In [1]:
from datasets import load_dataset

# eli5 = load_dataset("eli5", split="train_asks[:5000]")

data = load_dataset("DKYoon/SlimPajama-6B", streaming=True)

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

In [2]:
# eli5 = eli5.train_test_split(test_size=0.2)

In [2]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5", use_fast=False)

In [3]:
# eli5 = eli5.flatten()
# eli5["train"][0]

In [4]:
def preprocess_function(examples):
    # return tokenizer([" ".join(x) for x in examples["answers.text"]])
    return tokenizer([" ".join(x) for x in examples["text"]], truncation=True,
                padding="max_length",
                max_length=1024,
                # return_tensors="pt",
                )

In [7]:
# eli5.map(
#     preprocess_function,
#     batched=True,
#     num_proc=4,
#     remove_columns=eli5["train"].column_names,
# )

In [9]:
tokenized_eli5 = data.map(
    preprocess_function,
    batched=True,
    # num_proc=4,
    remove_columns=data["train"].column_names,
)

In [10]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True
                                # , num_proc=4
                                )

In [12]:
# train_data = iter(lm_dataset['train'])

In [13]:
# e_0 = next(train_data)
# e_1 = next(train_data)
# e_2 = next(train_data)

In [14]:
# len(e_0['input_ids']), len(e_0['labels']), len(e_0['attention_mask'])
# len(e_1['input_ids']), len(e_1['labels']), len(e_1['attention_mask'])

In [15]:
# lm_dataset['train'][0]['input_ids'] == lm_dataset['train'][0]['labels']
# lm_dataset['train'][45]['attention_mask']

In [16]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [17]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, LlamaModel, LlamaConfig, LlamaForCausalLM

# model = AutoModelForCausalLM.from_pretrained("distilgpt2")

import json

with open('configs.json', 'r') as file:
    config_dict = json.load(file)

# Create the configuration object
config = LlamaConfig.from_dict(config_dict)

# Initialize the LlamaForCausalLM model
model = LlamaForCausalLM(config)

In [18]:
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    max_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,

)

trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wel019. Use `wandb login --relogin` to force relogin


/home/shawn/anaconda3/envs/llm/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,10.262127


TrainOutput(global_step=100, training_loss=10.302589111328125, metrics={'train_runtime': 316.7961, 'train_samples_per_second': 20.202, 'train_steps_per_second': 0.316, 'total_flos': 5598294835200.0, 'train_loss': 10.302589111328125, 'epoch': 1.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 25043.49
